In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [3]:
#Extract data from pdf
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents

In [4]:
extracted_data = load_pdf("Data/")

In [5]:
# Text Chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size= 500,chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks


In [6]:
text_chunks = text_split(extracted_data)
print("Length of chunks",len(text_chunks))

Length of chunks 39994


In [7]:
# Download Embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [8]:
embeddings = download_hugging_face_embeddings()

/tmp/ipykernel_177838/253801913.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/media/vishal/Data3/LLM_LANG/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [10]:
query = embeddings.embed_query("hello world")
print("query",len(query))

query 384


In [ ]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

os.environ["PINECONE_API_KEY"] = "Your API Key Here"
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medical-chatbot"
print(f"Index Name: {index_name}")
re = PineconeVectorStore(
    embedding=embeddings,
    index_name=index_name  
)
docsearch = re.from_texts([t.page_content for t in text_chunks], embedding=embeddings, index_name=index_name)


Index Name: medical-chatbot


In [44]:
docsearch = re.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs = docsearch.similarity_search(query, k=3)

print("result:", docs)

result: [Document(id='e276a42c-d79f-42bd-ab4f-389ae4eedc09', metadata={}, page_content='‘‘What’s New in: Asthma and Allergic Rhinitis.’’Pulse\n(September 20, 2004): 50.\nRichard Robinson\nTeresa G. Odle\nAllergies\nDefinition\nAllergies are abnormal reactions of the immune\nsystem that occur in response to otherwise harmless\nsubstances.\nDescription\nAllergies are among the most common of medical\ndisorders. It is estimated that 60 million Americans, or\nmore than one in every five people, suffer from some\nform of allergy, with similar proportions throughout'), Document(id='0ade57db-fcbf-4297-8099-5444971ff61e', metadata={}, page_content='KEY TERMS\nAllergen— A substance capable of inducing an\nallergic response.\nAllergic reaction— An immune system reaction to\na substance in the environment; symptoms include\nrash, inflammation, sneezing, itchy watery eyes,\nand runny nose.\nConjunctiva— The mucous membrane that covers\nthe white part of the eyes and lines the eyelids.\nEdema— A co

In [46]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say you don't know, don't try to make up an answer.

Context = {context}
Question = {question}


Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [47]:
PROMPT = PromptTemplate(template=prompt_template, input_variables = ["context","question"])
chain_type_kwargs = {"prompt":PROMPT}

In [49]:
llm = CTransformers(model = "Model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={'max_new_tokens':512,
                            'temperature':0.8})

In [50]:
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type="stuff",
    retriever = docsearch.as_retriever(search_kwargs={'k':2}),
    return_source_documents= True,
    chain_type_kwargs = chain_type_kwargs
)

In [ ]:
while True:
    user_input = input(f"Input Prompt:")
    result = qa({"query":user_input})
    print("Response:", result["result"])

/tmp/ipykernel_177838/3967512645.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"query":user_input})


Response: Acne is a skin disorder that occurs when the sebaceous glands become inflamed.
